In [1]:
using JuMP, GLPK

<img src="example.png" style="width: 300px;"/>

Define the problem

In [2]:
trees = [
    (1,8),
    (2,4), (2,5),
    (3,2), (3,4),
    (4,8),
    (5,2), (5,7),
    (6,4),
    (7,8),
    (8,4), (8,7)
]

col_sums = [2,0,1,3,1,1,2,2]
row_sums = [2,0,4,0,2,1,1,2]
;

Initialize the solver

In [13]:
model = Model(with_optimizer(GLPK.Optimizer))

@variable(model, x[1:8, 1:8], Bin);

Add constraint for a tent to be next to a tree (but not on a tree)

In [14]:
# a tent must be next to a tree
for row in 1:8, col in 1:8
    # is not on a tree
    if (row, col) in trees
        @constraint(model, x[row, col] == 0)
    end
    
    # should be next to a tree 
    found = false
    for tree in trees
        if abs(tree[1] - row) + abs(tree[2] - col) <= 1
            found = true
        end
    end
    if !found
        @constraint(model, x[row, col] == 0)
    end
end

Add row and column sums

In [15]:
# columns and sums should add up
for i in 1:8
    @constraint(model, sum(x[i, :]) == row_sums[i])
    @constraint(model, sum(x[:, i]) == col_sums[i])
end

Solve

In [50]:
JuMP.optimize!(model)

solution = JuMP.value.(x)

println(" ----------------------------------- ")
for row in 1:8
    print("| ")
    for col in 1:8
        if solution[row, col] == 1
            print("⛺")
        elseif (row, col) in trees
            print("🌲")
        else
            print("◽")
        end
        print("  ")
    end
    println("|")
end
println(" ----------------------------------- ")

 ----------------------------------- 
| ◽  ◽  ◽  ⛺  ◽  ◽  ⛺  🌲  |
| ◽  ◽  ◽  🌲  🌲  ◽  ◽  ◽  |
| ⛺  🌲  ⛺  🌲  ⛺  ◽  ◽  ⛺  |
| ◽  ◽  ◽  ◽  ◽  ◽  ◽  🌲  |
| ⛺  🌲  ◽  ⛺  ◽  ◽  🌲  ◽  |
| ◽  ◽  ◽  🌲  ◽  ◽  ⛺  ◽  |
| ◽  ◽  ◽  ⛺  ◽  ◽  ◽  🌲  |
| ◽  ◽  ◽  🌲  ◽  ⛺  🌲  ⛺  |
 ----------------------------------- 
